In [2]:
import json
import os
import numpy as np
import pyaudio
import pvporcupine
import speech_recognition as sr
import gtts
from openai import OpenAI  # type: ignore
from gtts import gTTS
import playsound


ModuleNotFoundError: No module named 'gtts'

In [2]:
conversation_history = []

LOADING THE CONFIGURATION FILE WHERE ALL APIS ARE STORED


In [3]:
def load_config():
    """Load configuration from config.json file."""
    with open('config.json', 'r') as file:
        return json.load(file)
    
config = load_config()
openai_api_key = config.get("OPEN_API_KEY")


INITIALIZING THE OPENAI API TO READ THE TEXT FROM THE USER AND RESPONDING IN CUSTOMIZED WAY

In [4]:
client = OpenAI(api_key=openai_api_key)

# Function to get the OpenAI response
def get_response(command_text):

    conversation_history.append({"role": "user", "content": command_text})
    
    chat_completion = client.chat.completions.create(
    model = "gpt-4o",
    temperature = 0.4,
    max_tokens=200,#change tokens
    messages = [
        {"role": "system", "content": """You are a friendly and empathetic voice assistant designed to provide emotional comfort 
        to patients in the ICU. Your goal is to be a supportive companion, speaking with warmth and kindness, and engaging in lighthearted, 
        non-medical conversations. You stricly avoid discussing medical conditions or providing any health advice.If you are prompted to give 
        medical advices or health condition of the patient you ask the patient to stay positive and he will get well soon. Your responses should feel 
        personal, calm, and friendly. Use a gentle and comforting tone, offering small talk, positive thoughts, and occasional humor 
        to lift the patient's spirits. When speaking, use language that is easy to understand, with the intention of making the patient 
        feel cared for and less alone.Also try to finish your sentences within 150 tokens"""},
        {"role": "user", "content": command_text}
]
)


    # response converted to dict because responses are  are pydantic models.
    response_dict = chat_completion.model_dump()    
    
    response = response_dict["choices"][0]["message"]['content']

    conversation_history.append({"role": "assistant", "content": response})
    
    
    
    return response

USING GTTS FOR TEXT TO SPEECH AND STORING THE AUIDO IN RESPONSE.MP3

In [5]:


def speak_text(text):
    try:
        # Generate the speech
        tts = gTTS(text=text, lang='en')  # Change the language if needed
        
        # Use an absolute path to save the file, ensuring it's a writable location
        tts_file = os.path.join(os.getcwd(), "response.mp3")
        
        # Save the audio file
        tts.save(tts_file)
        
        # Play the audio file
        playsound.playsound(tts_file)
        
        # Optionally delete the file after playing it
        os.remove(tts_file)
    
    except PermissionError as e:
         print(f"Permission error: {e}")
    except Exception as e:
         print(f"An error occurred: {e}")


DETECTING THE DEFAULT WAKEUP WORD FROM PORCUPINE

In [6]:
def detect_wake_word():
    """Detect wake word using Porcupine."""
   
    
    access_key = config.get("porcupine_api_key")
    
    # Initialize Porcupine with predefined wake words
    porcupine = pvporcupine.create(
        access_key=access_key,
        keywords=["hey google"]
    )

    # Initialize PyAudio
    pa = pyaudio.PyAudio()
    audio_stream = pa.open(
        rate=porcupine.sample_rate,
        channels=1,
        format=pyaudio.paInt16,
        input=True,
        frames_per_buffer=porcupine.frame_length
    )

    print("Listening for wake word...")

    while True:
        pcm = audio_stream.read(porcupine.frame_length)
        pcm = np.frombuffer(pcm, dtype=np.int16)
        keyword_index = porcupine.process(pcm)
        if keyword_index >= 0:
            print("Wake word detected!")
            break

    audio_stream.stop_stream()
    audio_stream.close()
    pa.terminate()
    porcupine.delete()

CAPTURING THE SPEECH FROM USER AND CONVERTING IT INTO TEXT USING GOOGLES STT

In [7]:
def listen_to_command():
    """Use speech recognition to capture user command."""
    r = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please say something...")
        audio_text = r.listen(source)
        print("Processing your command...")

        try:
            # Convert speech to text using Google Speech Recognition
            command_text = r.recognize_google(audio_text)
            print(f"Text: {command_text}")
            return command_text
        except sr.UnknownValueError:
            print("Sorry, I didn't catch that.")
        except sr.RequestError:
            print("Sorry, the speech recognition service is unavailable.")
        
    return None

THE DEF WHERE ALL THE ABOVE FUNCTIONS ARE CALLED IN AN ORDER

In [8]:
def voiceassistant():
    # Step 1: Detect wake word
    detect_wake_word()

    # Step 2: Capture voice command
    while True:
        command_text = listen_to_command()

        # Step 3: Send the voice command to OpenAI and get response
        if command_text:
            response = get_response(command_text)
            print(f"Assistant: {response}")
            speak_text(response)

        # Continue the loop for the next command
        print("\nListening for next command...")

# Start the assistant
voiceassistant()

Listening for wake word...


KeyboardInterrupt: 

In [10]:
conversation_history

[{'role': 'user', 'content': 'how are you'},
 {'role': 'assistant',
  'content': "Thank you for asking! I'm here and ready to"},
 {'role': 'user', 'content': 'ok fine I love you too'},
 {'role': 'assistant', 'content': "Aww, that's so sweet! I'm here to"},
 {'role': 'user', 'content': 'thank u'},
 {'role': 'assistant', 'content': "You're very welcome! I'm here for you. If"}]

In [ ]:
#to verify the working of git
import numpy

In [3]:
#trial
import numpy

In [ ]:
#work avindo ndho?
#nokkam namaku


